In [52]:
#correlation matrix
library(FactoMineR)
library(RSQLite)
drv = SQLite()
conn = dbConnect(drv, dbname = './database')
# #getting max exp for genes where dev expression is highest, then bind to results for not dev expression
result <- dbGetQuery(conn, 'SELECT 
                              gene_features.id, 
                              gene_features.gLength,
                              gene_features.cds_len,  
                              gene_features.unique_domains, 
                              gene_features.domains, 
                              gene_features.motif_number, 
                              gene_features.gc3, 
                              gene_features.exp, 
                              gene_features.specificity,
                              gene_features.trans_count,
                              gene_features.intron_count,
                              gene_features.intron_length,
                              gene_features.avg_intron,
                              dn_ds,
                              dup_type
                            FROM 
                              gene_features
                            INNER JOIN 
                              macaque_paralogs 
                            ON 
                              gene_features.id == macaque_paralogs.id
                            WHERE 
                              macaque_paralogs.dn_ds 
                              AND macaque_paralogs.dn_ds < 10
                              AND Dev == "No"
                              AND (dup_type == "WGD" OR dup_type == "SSD")')[,-1]

extra <- dbGetQuery(conn, 'SELECT 
                            gene_features.id, 
                            gene_features.gLength,
                            gene_features.cds_len, 
                            gene_features.unique_domains, 
                            gene_features.domains, 
                            gene_features.motif_number,  
                            gene_features.gc3, 
                            gene_features.dev_exp, 
                            gene_features.specificity,
                            gene_features.trans_count,
                            gene_features.intron_count,
                            gene_features.intron_length,
                            gene_features.avg_intron,
                            dn_ds,
                            dup_type
                           FROM 
                            gene_features
                           INNER JOIN 
                             macaque_paralogs 
                            ON 
                             gene_features.id == macaque_paralogs.id
                           WHERE 
                             macaque_paralogs.dn_ds 
                             AND macaque_paralogs.dn_ds < 10
                             AND Dev == "Yes"
                             AND (dup_type == "WGD" OR dup_type == "SSD")')[,-1]
names(extra)[7] <- 'exp'

result <- rbind(result,extra)




result <- na.omit(result)
library("Hmisc")
res2 <- rcorr(as.matrix(result[,-14]))
corr <- res2$r
pval <- res2$P

write.csv(corr, file= 'corr.csv')
write.csv(pval, file= 'corr_pval.csv')


# for (i in 1:length(names(result)))
# {
#     for (j in 1:length(names(result)))
#     {
#         cor = cor.test(result[,i], result[,j], method = 'pearson')
#         print(names(result[i]))
#         print(names(result[j]))
#         print(cor$p.value)
#         print(cor$estimate['cor'])
#     }
# }

In [53]:
corr_results <- result[,c(1:4,10:12)]
#PCA
png('tester.png', width = 1000, height = 500)
pca_res <- PCA(corr_results, graph = F)
plot(pca_res)
# pca_res$eig
# pca_res$var$cor
dev.off()

pdf 
  2

In [54]:
PCA1 <- pca_res$ind$coord[,1]

PCA2 <- pca_res$ind$coord[,2]
rc <- rcorr(A,B)

In [55]:
non_corr <- result[,c(5:9,13,14)]

In [56]:
final_df <- cbind(non_corr, PCA1, PCA2)
# glm

In [58]:
for(i in 1:nrow(final_df))
{
    if(final_df[i,]['dup_type'] == 'WGD')
    {
        final_df[i,]['dup_type'] <- 1
    }
    if(final_df[i,]['dup_type'] == 'SSD')
    {
        final_df[i,]['dup_type'] <- 0
    }
}

In [190]:
model <- lm(dup_type ~ exp, data = final_df)

In [191]:
AIC(model)

[1] 13199.22

In [193]:
s <- summary(model)

In [194]:
for(i in 1:length(s$coefficients[,'Pr(>|t|)']))
{
    if(s$coefficients[,'Pr(>|t|)'][i] < 0.05)
    {
        print(row.names(s$coefficients)[i])
    }
}


[1] "(Intercept)"
[1] "exp"


In [195]:
s$coefficients['Estimate']

[1] NA

In [196]:
s$coefficients[,'Estimate']['dn_ds']

<NA> 
  NA

In [197]:
model_p

value 
    0

In [198]:
t_value

dn_ds 
-0.894679

In [199]:
std.error

dn_ds 
0.0790764

In [200]:
train <- final_df[1:5000,]
test <- final_df[5001:nrow(final_df),]
model <- lm(dup_type ~ exp, data = train)

In [201]:
testing <- predict(model, test)

In [202]:
RSS <- c(crossprod(model$residuals))
MSE <- RSS / length(model$residuals)
RMSE <- sqrt(MSE)

In [203]:
print(RMSE)

[1] 0.4990619
